In [1]:
import sqlite3
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

# Enable progress_apply for pandas
tqdm.pandas()

/home/tahlilmahfuz/RAG_Implementation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Load user reviews from the SQLite database
db_path = "Auto_Reply_v3.db"
conn = sqlite3.connect(db_path)
queries = [
    'SELECT "translation" AS Review, "Developer Reply Text" AS Reply FROM All_Reviews',
    'SELECT "translation" AS Review, "Reply" AS Reply FROM "Auto Reply Hadith"',
    'SELECT "translation" AS Review, "Reply" AS Reply FROM Auto_Reply_Reviews',
    'SELECT "translation" AS Review, "Reply" AS Reply FROM Auto_Reply_Reviews_Al_Quran'
]
dfs = [pd.read_sql_query(q, conn) for q in queries]
df = pd.concat(dfs, ignore_index=True)
conn.close()

In [3]:
# 2. Load the tokenizer and model for sequence classification
model_name = "iSathyam03/McD_Reviews_Sentiment_Analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [4]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Loaded '{model_name}' on {device}.")


Loaded 'iSathyam03/McD_Reviews_Sentiment_Analysis' on cuda.


In [6]:
# 3. Sentiment prediction function using logits
def predict_sentiment(text):
    if not isinstance(text, str) or not text.strip():
        return None
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}
    return sentiment_labels.get(prediction)

# 4. Apply sentiment prediction with progress bar
print("Starting sentiment analysis...")
df['Sentiment'] = df['Review'].progress_apply(predict_sentiment)
df

# 5. Save the results to CSV
# df.to_csv('McD_Reviews_Sentiment_Results.csv', index=False)
# print(f"Processed {len(df)} reviews. Results saved to 'McD_Reviews_Sentiment_Results.csv'.")

Starting sentiment analysis...


100%|██████████| 70460/70460 [04:45<00:00, 246.92it/s]


,Review,Reply,Sentiment
0,"Alhamdulillah.. the best al quran application,...",None,positive
1,the best quraan app you can find. it literally...,None,positive
2,Very helpful,None,positive
3,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...,negative
4,May Allah reward you richly rewarded,None,positive
...,...,...,...
70455,\tVery good app,BarakAllahu feekum for your kind words and sup...,positive
70456,"\tThank God, this application is really good!!...",BarakAllahu feekum for your kind words and sup...,positive
70457,"\tMashallah, may God bless you",BarakAllahu feekum for your kind words and sup...,positive
70458,\tThank God this application helps👍,BarakAllahu feekum for your kind words and sup...,positive


In [9]:
# 5. Generate automated replies based on sentiment

def generate_reply(sentiment):
    if sentiment == "positive":
        return (
            "BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!"
        )
    elif sentiment == "negative":
        return (
            "Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran."
        )
    else:
        return "JazakAllahu khayran for your review! If you face any issues or have feature requests, please provide your feedback on feedback.gtaf.org."

# Apply reply generation
df['Reply'] = df['Sentiment'].apply(generate_reply)
df

,Review,Reply,Sentiment
0,"Alhamdulillah.. the best al quran application,...",BarakAllahu feekum for your kind words and sup...,positive
1,the best quraan app you can find. it literally...,BarakAllahu feekum for your kind words and sup...,positive
2,Very helpful,BarakAllahu feekum for your kind words and sup...,positive
3,The recent edition does explain the source of ...,Assalamu 'alaykum dear user. We would love to ...,negative
4,May Allah reward you richly rewarded,BarakAllahu feekum for your kind words and sup...,positive
...,...,...,...
70455,\tVery good app,BarakAllahu feekum for your kind words and sup...,positive
70456,"\tThank God, this application is really good!!...",BarakAllahu feekum for your kind words and sup...,positive
70457,"\tMashallah, may God bless you",BarakAllahu feekum for your kind words and sup...,positive
70458,\tThank God this application helps👍,BarakAllahu feekum for your kind words and sup...,positive


In [10]:
# 6. Save the results to CSV including Review, sentiment, and reply
df.to_csv('4.McDSentimentAnalysis.csv', index=False)
print(f"Processed {len(df)} reviews. Results saved to '4.McDSentimentAnalysis.csv'.")

Processed 70460 reviews. Results saved to '4.McDSentimentAnalysis.csv'.
